# NOTEBOOK DE TRAITEMENT DES DONNEES DE RESSENTI 

In [1]:
%load_ext autoreload
%autoreload 2

In [388]:
import sys

sys.path.append(r"C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees")
sys.path.append(r"C:\Users\martin.schoreisz\git\Outils\Outils\src")
sys.path.append(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\Codes\src\Carac_situ_acoustiq"
)
sys.path.append(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\Codes\src"
)

import locale, os
from collections import Counter
from datetime import datetime, time, timedelta
from math import cos, degrees, log, log10, pi, pow, radians, sin, exp

import altair as alt
import numpy as np
import pandas as pd
from Bruit.Emission import Route
from Bruit.Meteo import (
    DonneesMeteo,
    calculCategorieVentQualitatif,
    calculConditionPropagationMeteoFranceQualitatif,
    calculGradientVitesseSon,
    conditionPropagation,
    dfAngle,
    dicoSecteurAngulaire,
    graphCompTempMesureeHautBas,
    secteurAngulaireQuantitatif,
)
from Bruit.Niveaux import calculLden, calculTNI, niveau2Pression, sommeEnergetique, moyenneQuadratiquePression, pression2Niveau, calculHarmonica, calculIntermitencyRatio
from Expoitation_mesures.Acoustique import recupDonneesAgregees, recupFractileAgregees, recupDonneesAcoustiqueSiteInstru, recupHarmonica
from Expoitation_mesures.Analyses_croises import (
    creerCorrelation,
    fusionnerDfSources,
    graphCroiseBruitMeteo1Jour,
    graphCroiseBruitMeteoPlusieursJour,
    graphCroiseBruitMeteoTrafic,
    graphCroiseBruitTrafic6min,
)
from Expoitation_mesures.Meteo import (
    concatMeteoFranceEtSite,
    creerGraphCompConditionsPropa,
    creerGraphCompVentSiteMeteoFrance,
    creerGraphMeteo,
    recupDonneesMeteoBase,
    recupDonneesMeteofranceBase,
)
from Exploitation_ressenti.Analyse_declaration import (
    filtrerDeclas,
    passageDeclas6min,
    nbParticipantParJour,
    graphNbVotantParJour,
    grouperDeclaParHorodate,
    graphEvolutionMoyMinMaxStd,
    creerChartDeclaHeureMinute,
    ajouterIndicateursTest,
    graphMoyennesPonderees,
    ajouterMoyennePonderee,
    graphHarmonica,
    graphComparaisonBruitDecla,
    nbParticipantParPasTemp,
    graphCorrelarionIndicTest
)
from Connexions.Connexion_Transfert import ConnexionBdd
from Expoitation_mesures.Trafic import *
from Import_stockage_donnees.Import_export_bdd import ResultatsEnquete
from Import_stockage_donnees.Params import dicoInstruSite
from Outils.Outils import dateTexteDepuisDayOfYear
from math import log10, pow, radians, pi, cos, sin, degrees

locale.setlocale(locale.LC_ALL, "fr_FR")
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_colwidth", 100)
# alt.data_transformers.disable_max_rows()
from toolz.curried import pipe
def altairDataDir(data, data_dir='altairData'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('altairDataDir', altairDataDir)
alt.data_transformers.enable('altairDataDir')

DataTransformerRegistry.enable('altairDataDir')

# ANALYSE DES DECLARATIONS DE GENE
Il s'agit de fournir des indicateurs de répartition des différentes valeurs de contexte et de déclaration fournies par les participants. Attention, les formulaires papiers ne permettaient pas de remplir tout les informations de contexte

<div class="alert alert-block alert-warning"><b>Attention:</b> Il n'y a pas d'analyse des commentaires réalisée pour le moment.</div>

## analyse globale sur les déclarations

In [9]:
resultats = ResultatsEnquete()
resultats.creationCharts(export=False)
# si export = True, alors les données sont excportées dans le dossier dossierExportChartsRessenti du module Params
# exemple : resultats.dicoCharts['localisation_gene']['chart']

In [10]:
# preparation des données
# on limite les données à la période de mesures et aux déclaration qui ne couvrent pas l'ensemble de la période
testChartDecla = filtrerDeclas(resultats)

In [21]:
len(testChartDecla), len(filtrerDeclas(resultats, dureeMaxiDeclaEnHeure=48))

(281, 283)

In [11]:
# Analyse visuelle des évolution de déclaration par jour
# Nombre de participants par jour
dfToutTemps = passageDeclas6min(testChartDecla)

## Recherche d'un indicateur global à comparer avec les niveaux de bruit
L'objectif est de trouver un indicateur qui permette de mettre en relation les niveaux de bruit mesurés et les déclarations. Pour pouvoir comparer avec les niveaux de bruit on va s'appuyer sur l'affectation des participants à un point fixe dans la Bdd, table ressenti.participants

In [12]:
# pour les graphs plus loin
slider = alt.binding_range(min=80, max=109, step=1)
select_day = alt.selection_single(
    name="date_gene", fields=["date_gene"], bind=slider, init={"date_gene": 80}
)

In [13]:
# Agrégation des données de façon à n'avoir qu'une seule ligne par par de 6 minutes
# on va calculer les indicateurs statistiques classiques : écart_type min, max, moyenne, nombre de vote
# on ne pernd pas la médiane, car les votes "extrême" reste des informatiosn à ne pas considérer comme déviantes
dfToutTempsIndics = grouperDeclaParHorodate(dfToutTemps)

In [14]:
# réprésentation graphiques de l'évolution de la moyenne, des min, max et du nombre de déclaration
graphEvolutionMoyMinMaxStd(dfToutTempsIndics)

alt.LayerChart(...)

### Analyse de l'évolution journalière des déclarations de gêne 1 
analyse simple basée sur la somme de toute les notes
$\sum \limits _{n = 0} ^{nbNote} note$

In [22]:
# exemple
slider = alt.binding_range(min=80, max=109, step=1)
select_day = alt.selection_single(
    name="date_gene", fields=["date_gene"], bind=slider, init={"date_gene": 80}
)
creerChartDeclaHeureMinute(dfToutTemps, hauteur=200).add_selection(
    select_day
).transform_filter(select_day)

alt.LayerChart(...)

In [23]:
# pour info, chart avec tous les jours et la méthode 1
# chart (par période de 6 linute et par jour) des déclarations et de la note cumulée
chart = alt.vconcat()
for i in range(80, 104, 2):
    dfFiltre1 = dfToutTemps.loc[dfToutTemps.date_gene == i]
    dfTempsGroupeNote1 = dfToutTemps.loc[dfToutTemps.date_gene == i]
    dfFiltre2 = dfToutTemps.loc[dfToutTemps.date_gene == i + 1] if i < 107 else None
    dfTempsGroupeNote2 = (
        dfToutTemps.loc[dfToutTemps.date_gene == i + 1] if i < 107 else None
    )
    chart &= (
        (creerChartDeclaHeureMinute(dfFiltre1) | creerChartDeclaHeureMinute(dfFiltre2))
        if i < 107
        else (creerChartDeclaHeureMinute(dfFiltre1, dfTempsGroupeNote1))
    )
chart

alt.VConcatChart(...)

### Analyse de l'évolution journalière des déclarations de gêne 2
Méthode associant à la méthode précédente un coefficient lié à la variabilité des déclaration. Cette variabilité est traduite par un inverse de l'écart type par rapport à une variabilité jugée suffisante, et fixée à 1,5.$$\frac{1.5}{\sigma} * \sum \limits _{n = 0} ^{nbNote} note$$ Cela ne fonctionne que si $\sigma$ est différents de 0.<br>Dans le cas contraire, cette formule s'écrit $$\frac{1.5}{1.5-(NbVal*0.1)} * \sum \limits _{n = 0} ^{nbNote} note$$

In [30]:
# création de l'attribut
dfMoyennePonderee = ajouterIndicateursTest(dfToutTempsIndics)
# visu
domainMax = 150
(
    alt.Chart(dfMoyennePonderee)
    .transform_fold(["moyenne", "note_pondere_std"])
    .mark_line()
    .encode(
        x=alt.X("date_heure:T"),
        y=alt.Y("value:Q", scale=alt.Scale(domain=(0, domainMax))),
        color="key:N",
    )
).add_selection(select_day).transform_filter(select_day)

alt.Chart(...)

### Comparaison des méthodes 1 et 2 

In [25]:
# ajout de l'attribut méthode 1 dans le df de la méthode 2
# fourni via un graph horizontal
(
    creerChartDeclaHeureMinute(dfToutTemps, hauteur=300)
    .add_selection(select_day)
    .transform_filter(select_day)
    | alt.Chart(dfMoyennePonderee)
    .transform_fold(["moyenne", "note_cumul", "note_pondere_std"])
    .mark_line()
    .encode(
        x=alt.X("date_heure:T"),
        y=alt.Y("value:Q", scale=alt.Scale(domain=(0, domainMax))),
        color="key:N",
    )
    .add_selection(select_day)
    .transform_filter(select_day)
).resolve_legend("independent")

alt.HConcatChart(...)

<div class="alert alert-block alert-info">
La méthode de cumul des note produit un rendu fidèle à la fois du nombre de déclarant et de la valeur des notes. Cependant, cela ne fournit pas d'information sur la dispersion des valeurs ou le nombre de votant à un instant précis. De plus, l'échelle de lecture n'est pas commune avec le système de note.  
La méthode de note pondérée présente le même défaut d'échelle par rapport au note, mais apporte un éclairage sur la dispersion (exemple le jour 80), ou le niveau pondéré passe sous le niveau cumulé, du fait de la présence d'une note de faible valeur. Cependant, cela crée des aussides effets de bords lorsque des notes élevées s'arretent. En effet, dans ce cas la dispersion diminue, donc la pondération augmente ; par exemple le jour 85, à partir de 19h, certaines augmentations correspondent à des déclarartions de niveaux élevées qui s'arretent
</div>

### Analyse de l'évolution journalière des déclarations de gêne 3
Méthode basée sur une moyenne pondérée( $ \frac{\sum\limits _{n = 0} ^{10} note * NbApparitionDeLaNote}{\sum\limits _{n = 0} ^{10} NbApparitionDeLaNote}  $) .<br> En L'ajout d'une fonction du nombre de votant pour cette note par rapport à un seuil arbitraire permettant de tester si la masse de votant fourni une inforation supplémentaire :
$\frac{\sum\limits _{n = 0} ^{10} (note + \ln(\frac{NbApparitionDeLaNote}{x})) * (NbApparitionDeLaNote^x)}{\sum\limits _{n = 0} ^{10} NbApparitionDeLaNote^x} $ avec $x =$ Nombre de votant considéré comme représentatifs .<br> l'utilisation du logarithme du $\frac{NbApparitionDeLaNote}{nbVotantConsidéréCommeReprésentatifs}$ permet de créer une notation moins importante que la note fournie pour les notes avec peu de déclarant, et une note plus importante pour les notes fournies avec bcp de déclarants

In [36]:
(
    creerChartDeclaHeureMinute(dfToutTemps, hauteur=300)
    .add_selection(select_day)
    .transform_filter(select_day)
    &
    graphMoyennesPonderees(dfMoyennePonderee, listAttributs=["moyenne", "moy_pond", "moy_pond_sur_k"]).add_selection(select_day).transform_filter(select_day)
)

alt.VConcatChart(...)

Les moyennes pondérées fournissent un indicateur à l'échelle des notes. Elles permettent de donner plus de poids aux notes les plus votées. la différenciation entre des périodes ayant une unique note fournie par n personnes, et les périodes ayant la même unique note, fournie par n + x personnes est bien traitée apr l'ajout du terme correctif $\ln(\frac{NbApparitionDeLaNote}{x})$. Par exemple, dans le cas où 2 périodes présente une moyenne de 10 (note maximale), l'une avec un seul vote, l'autre avec 5 votes, en fixant une note comme représentative à partir de plus de 2 votes, alors la pondération avant moyenne va permettre de transformer les valeurs 10 en $10 + \ln(\frac{1}{2})$ et $10 + \ln(\frac{5}{2})$, soit 9,3 et 10,9 : dans ce cas, la pondération directe des notes permet de différencier les moyennes de fces deux situations.

In [27]:
dfMoyennePonderee = ajouterMoyennePonderee(dfToutTempsIndics)

# ANALYSE CROISÉE BRUIT - RESSENTI

## analyse LAeq et indicateur ressenti

In [11]:
site = 4

In [12]:
# pas temporel de 6 minutes
# récupérationdes données de bruit
dfBruit6Min = recupDonneesAgregees(site, "6 min")
# récupération des donnée de ressenti
dfDecla6Min = passageDeclas6min(filtrerDeclas(ResultatsEnquete(), site=site))
dfRessenti6min = ajouterIndicateursTest(grouperDeclaParHorodate(dfDecla6Min))
# nombre de déclarant par jour
dfNbParticipantParJour = nbParticipantParJour(dfDecla6Min)
dfNbParticipantParJour6min = nbParticipantParPasTemp(dfNbParticipantParJour, '6min').reset_index(names='date_heure')
# ajout attributs temporels communs
for d in (dfBruit6Min, dfRessenti6min, dfNbParticipantParJour6min):
    d["heure_minute"] = d.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}")
    d["jour"] = d.date_heure.apply(lambda x: x.strftime("%A %d %B %Y"))
    d["jour_sort"] = d.date_heure.dt.dayofyear
    d["epoch"] = d.date_heure.apply(lambda x: x.timestamp())

In [13]:
# GRAPH
graphComparaisonBruitDecla(dfBruit6Min=dfBruit6Min, dfNbParticipantParJour=dfNbParticipantParJour6min, dfRessenti6min=dfRessenti6min,
                           title=['Comparaison Niveau de bruit et déclarations de gêne ;', 'Rue Jules Ladoumègue ; pas temporel = 6 min'])

alt.VConcatChart(...)

In [16]:
# pas temporel d'1h
# récupérationdes données de bruit
dfBruit1h = recupDonneesAgregees(site, "1 h")
# récupération des donnée de ressenti
dfDecla1h = passageDeclas6min(filtrerDeclas(ResultatsEnquete(), site=site), pasTemporel='1h')
dfRessenti1h = ajouterIndicateursTest(grouperDeclaParHorodate(dfDecla1h))
# nombre de déclarant par jour
dfNbParticipantParJour1h = nbParticipantParPasTemp(dfNbParticipantParJour, '1h').reset_index(names='date_heure')
for d in (dfBruit1h, dfRessenti1h, dfNbParticipantParJour1h):
    d["heure_minute"] = d.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}")
    d["jour"] = d.date_heure.apply(lambda x: x.strftime("%A %d %B %Y"))
    d["jour_sort"] = d.date_heure.dt.dayofyear
    d["epoch"] = d.date_heure.apply(lambda x: x.timestamp())

In [60]:
# croisement des df pour corrélation
dfCroise1hLeq_a = dfBruit1h[['date_heure', 'leq_a']].merge(dfRessenti1h[['date_heure', 'moy_pond_sur_k', 'note_cumul']], on='date_heure', how='left').merge(
    dfNbParticipantParJour1h[['date_heure', 'nb_participants']], on='date_heure', how='left'
)
dfCroise1hLeq_a = dfCroise1hLeq_a.loc[(dfCroise1hLeq_a.date_heure < '2022-04-13') & (dfCroise1hLeq_a.moy_pond_sur_k.notna()) & (dfCroise1hLeq_a.nb_participants.notna())]

In [61]:
graphComparaisonBruitDecla(dfBruit6Min=dfBruit1h, dfNbParticipantParJour=dfNbParticipantParJour1h, dfRessenti6min=dfRessenti1h,
                           title=['Comparaison Niveau de bruit et déclarations de gêne ;', 'Rue Jules Ladoumègue ; pas temporel = 1 h'])

alt.VConcatChart(...)

In [62]:
graphCorrelarionIndicTest(dfCroise1hLeq_a, 'leq_a')

alt.HConcatChart(...)

La corrélation entre le niveau de bruit LAeq et les déclarations de gênes varie fortement selon l'indicateur de gênes étudié et le site. La prise en compte de la période 6h-22h uniquement diminue systématiquement la corrélation entre les niveaux de bruit mesurés et la moyenne pondérée et la note cumulée. Le nombre de déclarant est un bon indicateur pour le site de mesure rue Jules Ladoumègue, mais ne correspond pas du tout pour le site de la Rue des Noyers. À l'inverse, la moyenne pondérée propose une corrélation supérieur sur le site de la Rue des Noyers, mais reste toujours inférieure ou proche de 0,3.<br> Au final, seule la note cumulée présente une corrélation relativement stable quelle que soit la configuration étudiée (site, pas temporel, période concernée), mais la valeur absolue de cette corrélation reste faible, ce qui tend à prouver que les déclaration de ressenti et le niveaux de bruit sont difficelemnt corrélable.

## Analyse de l'indicateur harmonica

In [20]:
# recupération des données harmonica pour un site
dfHarmonica = recupHarmonica(site=site)
# visualisation
graphHarmonica(dfHarmonica)

alt.LayerChart(...)

In [21]:
# comparaison avec la LAeq
dfHoraire = recupDonneesAgregees(site, "1 h")
dfHoraire = dfHoraire.assign(
    heure_minute=dfHoraire.date_heure.apply(lambda x: f"{x.strftime('%H:%M')}"),
    jour=dfHoraire.date_heure.apply(lambda x: x.strftime("%A %d %B %Y")),
    jour_sort=dfHoraire.date_heure.dt.dayofyear,
) 

In [22]:
# variation de Harmonica par rapport au LeqA
(alt.Chart(dfHarmonica).mark_line(color='blue').encode( 
    x='date_heure',
    y=alt.Y('harmonica', scale=alt.Scale(domain=(0,10), clamp=True), axis=alt.Axis(labelColor='blue',titleColor='blue' ))
)+alt.Chart(dfHoraire).mark_line(color='orange').encode( 
    x='date_heure',
    y=alt.Y('leq_a', scale=alt.Scale(domain=(45,80), clamp=True), axis=alt.Axis(labelColor='orange',titleColor='orange' ))
)).resolve_scale(y='independent').properties(width=600)

alt.LayerChart(...)

In [23]:
alt.Chart(dfHoraire).mark_line().encode( 
    x='date_heure',
    y='leq_a'
).properties(width=600)

alt.Chart(...)

In [63]:
# croisement des df pour corrélation
dfCroise1hHarmonica = (
    dfHarmonica[["date_heure", "harmonica"]]
    .merge(
        dfRessenti1h[["date_heure", "moy_pond_sur_k", "note_cumul"]],
        on="date_heure",
        how="left",
    )
    .merge(
        dfNbParticipantParJour1h[["date_heure", "nb_participants"]],
        on="date_heure",
        how="left",
    )
)
dfCroise1hHarmonica = dfCroise1hHarmonica.loc[
    (dfCroise1hHarmonica.date_heure < "2022-04-13")
    & (dfCroise1hHarmonica.moy_pond_sur_k.notna())
    & (dfCroise1hHarmonica.nb_participants.notna())
]

In [64]:
graphCorrelarionIndicTest(dfCroise1hHarmonica, 'harmonica')

alt.HConcatChart(...)

L'indicateur Harmonica, basé sur l'analyse du bruit de fond et des émergences par comparaison entre le LAeq et le 95ème percentile des niveaux de mesurés, ne fournit pas une meilleure corrélation avec le ressenti que le LAeq. Cela est logique, au vu des tests mené sur les niveaux fractiles précédemment, notamùment l'étude de l'indicateur TNI. 

## L50_1khz
L'indicateur LA50 est mentionné dans l'étude Grafic de l'UGE et l'université du 92 (ref ?) comme un dex indicateurs les plus corrélé avec le bruit du trafic routier

In [ ]:
site = 4

In [239]:
# recup des données
with ConnexionBdd(bdd) as c:
        dfTo1kHz = pd.read_sql(
            f"SELECT id_site, id, date_heure, to_1000 FROM mesures_physiques.niveau_bruit_site WHERE id_site = {site} ORDER BY date_heure",
            c.sqlAlchemyConn,
        )
Leq_a = recupDonneesAgregees(site=site, typeAgreg='1 h')
dfTo1kHz_1s = dfTo1kHz.set_index('date_heure').resample('1s').to_1000.apply(lambda x: pression2Niveau(moyenneQuadratiquePression([niveau2Pression(y) for y in x])))
dfTo1kHz_1h = dfTo1kHz_1s.reset_index().set_index('date_heure').resample('1h').to_1000.quantile(0.5)
dfCompLaeq_l50_wide = dfTo1kHz_1h.reset_index().merge(Leq_a, on = 'date_heure')


In [277]:
dfCompLaeq_l50_wide


,date_heure,to_1000,pression,leq_a,id_instru_site,id,periode_agreg
0,2022-03-21 13:00:00,55.948832,0.028912,63.200975,4,47139,1 h
1,2022-03-21 14:00:00,57.464776,0.029206,63.288965,4,47140,1 h
2,2022-03-21 15:00:00,56.647626,0.033444,64.465840,4,47141,1 h
3,2022-03-21 16:00:00,57.353437,0.030478,63.659191,4,47142,1 h
4,2022-03-21 17:00:00,55.864541,0.032686,64.266751,4,47143,1 h
...,...,...,...,...,...,...,...
689,2022-04-19 06:00:00,49.916535,0.010521,54.420706,4,47828,1 h
690,2022-04-19 07:00:00,50.439243,0.015771,57.936723,4,47829,1 h
691,2022-04-19 08:00:00,48.176796,0.014433,57.166480,4,47830,1 h
692,2022-04-19 09:00:00,48.321180,0.011563,55.241069,4,47831,1 h


In [273]:
# comparaison des indicateurs
alt.Chart(dfCompLaeq_l50_wide).transform_fold(['to_1000', 'leq_a']).mark_line().encode(
    x='date_heure',
    y=alt.Y('value:Q', scale=alt.Scale(domain=(40, 70), clamp=True)),
    color='key:N'
).properties(width=1200)

alt.Chart(...)

In [278]:
dfCompLaeq_l50_wide['diffLeq_L501khz'] = abs(dfCompLaeq_l50_wide.to_1000 - dfCompLaeq_l50_wide.leq_a)
dfCompLaeq_l50_wide.std(numeric_only=True)

to_1000            4.364454e+00
pression           6.631981e-03
leq_a              3.617774e+00
id_instru_site     4.444095e-16
id                 2.004848e+02
diffLeq_L501khz    2.442251e+00
dtype: float64

L'écart type de la différence entre le Laeq et le L50 à 1Khz est de 2.5 dB, avec un Leq_a systématiquement supérieur. Le leq_a semble donc plus pertinent sur ce secteur.

In [322]:
# croisement des df pour corrélation
dfCroise1hL501khz = dfCompLaeq_l50_wide[['date_heure', 'to_1000']].merge(dfRessenti1h[['date_heure', 'moy_pond_sur_k', 'note_cumul']], on='date_heure', how='left').merge(
    dfNbParticipantParJour1h[['date_heure', 'nb_participants']], on='date_heure', how='left'
)
dfCroise1hIR = dfCroise1hL501khz.loc[(dfCroise1hL501khz.date_heure < '2022-04-13') & (dfCroise1hL501khz.moy_pond_sur_k.notna()) & (dfCroise1hL501khz.nb_participants.notna())]

In [325]:
graphCorrelarionIndicTest(dfCroise1hL501khz, 'to_1000')

alt.HConcatChart(...)

Le L50 à 1 Khz présente une corrélation du même ordre que le LAeq, ce qui est logique au vu de la très forte correspondance d'évolution de ces deux indicateurs.

## TFSD / étude grafic
L'indicateur TFSD est issu de l'étude Grafic de l'UGE et l'université du 92 (ref ?). Il permet de quantifier la présence de voix humaines et cri d'oiseau pendant une période, à partir de fichier wav.
L'extraction des donnée par pas de 6 minutes a été réalisée via R, dans le [notebook suivant](https://github.com/nantodevison/Ech24/blob/main/notebooks/R_seewave.ipynb)

In [93]:
# Pour information : ouverture, agregation des données et transfert en Bdd
dossier = r"C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\TFSD"
listFichier = []
with os.scandir(dossier) as it:
    for entry in it:
        if entry.name.endswith('.csv') and entry.is_file():
            listFichier.append(os.path.join(dossier, entry.name))
dfTfsd = pd.concat([pd.read_csv(f) for f in listFichier])

In [29]:
alt.Chart(dfTfsd.loc[dfTfsd.site == 'JulesLadoumegue']).transform_density(
    'tsfd',
    as_=['tsfd', 'density'],
).mark_area().encode(
    x="tsfd:Q",
    y='density:Q',
)

alt.Chart(...)

In [121]:
# obtenir un tfsd moyen par heure pour un site
dfTfsdLadoumegue = dfTfsd.loc[dfTfsd.site == 'JulesLadoumegue'].copy()
dfTfsdLadoumegue['date_heure_base'] = dfTfsdLadoumegue.fichier.apply(lambda x: pd.to_datetime(f'{x[:4]}-{x[4:6]}-{x[6:8]} {x[9:11]}:{x[11:13]}:{x[13:15]}'))
dfTfsdLadoumegue['date_heure_deb'] = dfTfsdLadoumegue.apply(lambda x: x.date_heure_base + pd.Timedelta(seconds=int(x.periode.split('_')[0])-1), axis=1)
dfTfsdLadoumegue['date_heure_fin'] = dfTfsdLadoumegue.apply(lambda x: x.date_heure_base + pd.Timedelta(seconds=int(x.periode.split('_')[1])), axis=1)
dfTfsdLadoumegue_1h = dfTfsdLadoumegue[['date_heure_deb', 'tsfd']].set_index('date_heure_deb').resample('1h').max()
alt.Chart(dfTfsdLadoumegue_1h.reset_index()).mark_line().encode(
    x='date_heure_deb',
    y='tsfd'
)

alt.Chart(...)

les résultats indique une densité centrée autour de 0.025. Une valeur supéreir à 0.3 traduit une présence marquée de chants d'oiseaux ou de voix humaines. il apparit donc que sur ces secteurs, il n'y a quasiment de présence de voie ou bruit d'oiseaux

In [315]:
# on va chercher à calculer la note Graphic
# recup du dfTo1kHz_1h dans le chapitre précédent
width = 1200
height = 200
dfTfsd_l501Khz = dfTfsdLadoumegue_1h.reset_index().merge(
    dfCompLaeq_l50_wide, left_on="date_heure_deb", right_on="date_heure"
)
dfTfsd_l501Khz["agrementSonore"] = (
    16.54 - (0.25 * dfTfsd_l501Khz.to_1000) + (15 * dfTfsd_l501Khz.tsfd)
)
(
    alt.Chart(dfTfsd_l501Khz)
    .mark_line(color="orange")
    .encode(
        x="date_heure", y=alt.Y("tsfd:Q", scale=alt.Scale(domain=(0, 0.5), clamp=True))
    )
    .properties(width=width, height=height)
    & alt.Chart(dfTfsd_l501Khz)
    .mark_line(color="green")
    .encode(
        x="date_heure",
        y=alt.Y("to_1000:Q", title='L50 1KHz', scale=alt.Scale(domain=(45, 60), clamp=True)),
    )
    .properties(width=width, height=height)
    & alt.Chart(dfTfsd_l501Khz)
    .mark_line()
    .encode(x="date_heure_deb", y=alt.Y("agrementSonore", scale=alt.Scale(domain=(0, 10), clamp=True)))
    .properties(width=width, height=height)
)

alt.VConcatChart(...)

In [320]:
# croisement des df pour corrélation
# dans le cas de l'agrement sonore, plus la note est haute meilleur est le resultat. Comm ec'est une base 10, on va prendre la laveur aboslue pour k'inverser et regarder la correlation
dfTfsd_l501Khz['agrementSonoreInverse'] = abs(10-dfTfsd_l501Khz.agrementSonore)
dfCroise1hAG = dfTfsd_l501Khz[['date_heure', 'agrementSonoreInverse']].merge(dfRessenti1h[['date_heure', 'moy_pond_sur_k', 'note_cumul']], on='date_heure', how='left').merge(
    dfNbParticipantParJour1h[['date_heure', 'nb_participants']], on='date_heure', how='left'
)
dfCroise1hIR = dfCroise1hAG.loc[(dfCroise1hAG.date_heure < '2022-04-13') & (dfCroise1hAG.moy_pond_sur_k.notna()) & (dfCroise1hAG.nb_participants.notna())]

In [321]:
graphCorrelarionIndicTest(dfCroise1hAG, 'agrementSonoreInverse')

alt.HConcatChart(...)

L'ajout du TSFD au L50 à 1KHz permet d'améliorer légèrement la corrélation entre les indicateurs de ressenti et les indicateurs pohysiques, cependant cette corrélation n'atteint toujours pas les 0.5.

## Analyse de l'intermitency ratio

In [31]:
# recup des données des niveaus sonore sur le site
dFBrute = recupDonneesAcoustiqueSiteInstru(site)
# DataFrame de l'IR par pas de 1h
IR = [
    (calculIntermitencyRatio(
        dFBrute.loc[(dFBrute.date_heure >= j[0]) & (dFBrute.date_heure < j[1])]
    ),j[0])
    for j in [
        (d, d + pd.Timedelta("1H"))
        for d in pd.date_range("2022-03-21 12:00:00", "2022-04-17 23:00:00", freq="1H")
    ]
]
dfIR = pd.DataFrame(IR, columns=('IR', 'date_heure'))

In [32]:
# réprésentation graphique rapide
alt.Chart(dfIR).mark_line().encode(
    x='date_heure',
    y=alt.Y('IR', scale=alt.Scale(domain=(0,100)))
).properties(width=1200)

alt.Chart(...)

In [65]:
# croisement des df pour corrélation
dfCroise1hIR = dfIR[['date_heure', 'IR']].merge(dfRessenti1h[['date_heure', 'moy_pond_sur_k', 'note_cumul']], on='date_heure', how='left').merge(
    dfNbParticipantParJour1h[['date_heure', 'nb_participants']], on='date_heure', how='left'
)
dfCroise1hIR = dfCroise1hIR.loc[(dfCroise1hIR.date_heure < '2022-04-13') & (dfCroise1hIR.moy_pond_sur_k.notna()) & (dfCroise1hIR.nb_participants.notna())]

In [66]:
graphCorrelarionIndicTest(dfCroise1hIR, 'IR')

alt.HConcatChart(...)

De base, l'IR n'est pas correlé avec le ressenti, ce qui est normal puisque plus le bruit est stable, plus l'IR est faible, tandis que ce qui ressort de l'analyse de ressenti est plutot directement lié au niveau de bruit : plus le bruit est fort et stable, plus le ressenti est important.
Cela signifie donc, que le comportement de IR dans notre cas est inverse avec celui du ressenti. On va donc faire un premier test  pour voir ce que donne la corrélation en considérant l'inverse de IR

In [36]:
# calcul de IRInverse
dfIR['IRInverse'] = 100 - dfIR.IR

In [67]:
# croisement des df pour corrélation
dfCroise1hIRInverse = dfIR[['date_heure', 'IRInverse']].merge(dfRessenti1h[['date_heure', 'moy_pond_sur_k', 'note_cumul']], on='date_heure', how='left').merge(
    dfNbParticipantParJour1h[['date_heure', 'nb_participants']], on='date_heure', how='left'
)
dfCroise1hIRInverse = dfCroise1hIRInverse.loc[(dfCroise1hIRInverse.date_heure < '2022-04-13') & (dfCroise1hIRInverse.moy_pond_sur_k.notna()) & (dfCroise1hIRInverse.nb_participants.notna())]

In [68]:
graphCorrelarionIndicTest(dfCroise1hIRInverse, 'IRInverse')

alt.HConcatChart(...)

### tes avec un IR sur un seuil de arbitraire de 60dB(A)  
On teste plusieurs seuil proches des 60, cela provient de l'analyse des déclarations de gêne des riverains

In [356]:
IRSeuil = [
    (calculIntermitencyRatio(
        dFBrute.loc[(dFBrute.date_heure >= j[0]) & (dFBrute.date_heure < j[1])], C=None, seuilForce=sf
    ),j[0], sf)
    for j in [
        (d, d + pd.Timedelta("1H"))
        for d in pd.date_range("2022-03-21 12:00:00", "2022-04-13 23:00:00", freq="1H")
    ] for sf in [58,60,62,65]
]
dfIRSeuil = pd.DataFrame(IRSeuil, columns=('IRSeuil', 'date_heure', 'seuil'))

In [357]:
# réprésentation graphique rapide
alt.Chart(dfIRSeuil).mark_line().encode(
    x='date_heure',
    y=alt.Y('IRSeuil', scale=alt.Scale(domain=(0,100))),
    color='seuil:N'
).properties(width=1200)

alt.Chart(...)

In [358]:
# croisement des df pour corrélation
dfCroise1hIRSeuil = dfIRSeuil[['date_heure', 'IRSeuil', 'seuil']].merge(dfRessenti1h[['date_heure', 'moy_pond_sur_k', 'note_cumul']], on='date_heure', how='left').merge(
    dfNbParticipantParJour1h[['date_heure', 'nb_participants']], on='date_heure', how='left'
)
dfCroise1hIRSeuil = dfCroise1hIRSeuil.loc[(dfCroise1hIRSeuil.date_heure < '2022-04-14') & (dfCroise1hIRSeuil.moy_pond_sur_k.notna()) & (dfCroise1hIRSeuil.nb_participants.notna())]

In [359]:
((graphCorrelarionIndicTest(dfCroise1hIRSeuil.loc[dfCroise1hIRSeuil.seuil == 58], 'IRSeuil') | 
 graphCorrelarionIndicTest(dfCroise1hIRSeuil.loc[dfCroise1hIRSeuil.seuil == 60], 'IRSeuil')) & 
 (graphCorrelarionIndicTest(dfCroise1hIRSeuil.loc[dfCroise1hIRSeuil.seuil == 62], 'IRSeuil') | 
 graphCorrelarionIndicTest(dfCroise1hIRSeuil.loc[dfCroise1hIRSeuil.seuil == 65], 'IRSeuil')))

alt.VConcatChart(...)

L'IR au seuil de 60 donne des résultats bien plus correct que l'IR normal, proche de ceux du LAeq. Cependant, le taux de corrélation reste faible en valeur absolue, donc on va pouvoir essayer de combiner les deux pour voir si on peut obtenir un indicateur plus fiable.

## Combinaison de LAeq et IRseuil60

L'idée : le Laeq donne un résultat pas trop mal, L'IR au seil 60 aussi. L'IR seuil 60 traduit l'énergie sonore fournie par les évènements  de plus de 60 dB sur la période considérée, par rapport à l'énergie totale. Donc, si l'IRseuil60 est à 100%, cela veut dire que sur la période considérée, toute l'énergie sonore vient de niveaux de bruit de plus de 60 dB. 

In [360]:
# fusionner le la_eq et l'IRSeuil60
dfLaeqIr = dfCroise1hLeq_a.merge(dfIRSeuil, on='date_heure')
# calcul du leq_majore
dfLaeqIr['pression_base'] = dfLaeqIr.leq_a.apply(lambda x: niveau2Pression(x))
dfLaeqIr['pression_majoree'] = 0.01 * dfLaeqIr.IRSeuil * dfLaeqIr.pression_base
dfLaeqIr['niveau_corrige'] = dfLaeqIr.pression_majoree.apply(lambda x: pression2Niveau(x))
dfLaeqIr['leq_majore'] = dfLaeqIr.apply(lambda x: sommeEnergetique(pression2Niveau(x.pression_majoree), pression2Niveau(x.pression_base)), axis=1)

In [361]:
((graphCorrelarionIndicTest(dfLaeqIr.loc[dfLaeqIr.seuil == 58], 'leq_majore') | 
 graphCorrelarionIndicTest(dfLaeqIr.loc[dfLaeqIr.seuil == 60], 'leq_majore')) & 
 (graphCorrelarionIndicTest(dfLaeqIr.loc[dfLaeqIr.seuil == 62], 'leq_majore') | 
 graphCorrelarionIndicTest(dfLaeqIr.loc[dfLaeqIr.seuil == 65], 'leq_majore')))

alt.VConcatChart(...)

In [362]:
(alt.Chart(dfLaeqIr).transform_fold(['leq_a', 'note_cumul']).mark_line().encode(
    x='date_heure',
    y='value:Q',
    color='key:N'
) + alt.Chart(dfLaeqIr).mark_line().encode(
    x='date_heure',
    y='leq_majore:Q',
    color='seuil:N'
)).properties(width=1200, height=400)

alt.LayerChart(...)

La combinaison entre LAeq et IR ne permet de pas de gagner siginificativement en corrélation entre les déclarations de gêne et les indicateurs de niveaux sonores. Néanmoins, la meilleure combinaison possible est la majoration du LAeq par le l'IR  seuil à 60 dB.

### Test IR sur les seuils OMS

In [343]:
IRSeuilOMSJour = [
    (calculIntermitencyRatio(
        dFBrute.loc[(dFBrute.date_heure >= j[0]) & (dFBrute.date_heure < j[1])], C=None, seuilForce=53
    ),j[0], 53)
    for j in [
        (d, d + pd.Timedelta("1H"))
        for d in pd.date_range("2022-03-21 12:00:00", "2022-04-13 23:00:00", freq="1H")
    if d.hour >= 6 and d.hour<22]
]

In [344]:
IRSeuilOMSNuit = [
    (calculIntermitencyRatio(
        dFBrute.loc[(dFBrute.date_heure >= j[0]) & (dFBrute.date_heure < j[1])], C=None, seuilForce=45
    ),j[0], 45)
    for j in [
        (d, d + pd.Timedelta("1H"))
        for d in pd.date_range("2022-03-21 12:00:00", "2022-04-13 23:00:00", freq="1H")
    if d.hour < 6 or d.hour >= 22]
]

In [347]:
dfIRSeuilOMS = pd.concat([pd.DataFrame(IRSeuilOMSJour, columns=('IRSeuil', 'date_heure', 'seuil')), pd.DataFrame(IRSeuilOMSNuit, columns=('IRSeuil', 'date_heure', 'seuil'))])

In [349]:
# réprésentation graphique rapide
alt.Chart(dfIRSeuilOMS).mark_line().encode(
    x='date_heure',
    y=alt.Y('IRSeuil', scale=alt.Scale(domain=(0,100)))
).properties(width=1200)

alt.Chart(...)

In [351]:
# croisement des df pour corrélation
dfCroise1hIRSeuilOMS = dfIRSeuilOMS[['date_heure', 'IRSeuil', 'seuil']].merge(dfRessenti1h[['date_heure', 'moy_pond_sur_k', 'note_cumul']], on='date_heure', how='left').merge(
    dfNbParticipantParJour1h[['date_heure', 'nb_participants']], on='date_heure', how='left'
)
dfCroise1hIRSeuilOMS = dfCroise1hIRSeuilOMS.loc[(dfCroise1hIRSeuilOMS.date_heure < '2022-04-14') & (dfCroise1hIRSeuilOMS.moy_pond_sur_k.notna()) & (dfCroise1hIRSeuilOMS.nb_participants.notna())]

In [352]:
graphCorrelarionIndicTest(dfCroise1hIRSeuilOMS, 'IRSeuil')

alt.HConcatChart(...)

L'utilisation des valeurs limites OMS comme seuil de dépassement du calcul de l'IR n'est pas probante. Cela s'explique par le caractère continu du dépassement de valeur limite : aux abords de la rocade, le bruit est continu. De fait, l'IR avec seuil aux valeurs OMS est quasi constant à 100%, tandis que les indicateurs issus des déclarations de gêne varient. Cela permet donc d'affirmer que le niveau de bruit seuil au-delà duquel les riverains se sentent gêné se situe plus aux alentours de 60 dB que des seuisl OMS.

## test global avec Leq, TSFD et IRseuil60
Attention, il faut avoir joué toute la partie indicateur croisé bruit-ressenti en amont

In [461]:
# dans unpremier temp on trouve une fonction d'ajout / retrait de niveau de bruit à LAeq en fonction de TSFD.
# le principe : si TSFD est < 0.1 c'est que pas de bruit sympa, donc on va ajouter en plus un malus si le niveau de bruit n'est pas calme
# un graph pour la fonction de bonus/malus propre au TSFD
alt.Chart(pd.DataFrame([(x/1000, log10((x)/1000)*-1 if x/100 <= 1 else exp(2*x/1000)*-1) for x in range(1, 1000, 5)], columns=('x', 'log10'))).mark_line().encode(
    x='x',
    y=alt.Y('log10', scale=alt.Scale(domain=(-5,5), clamp=True))
)
dfLaeqIr

alt.Chart(...)

In [465]:
dfTotIndic = dfLaeqIr.loc[dfLaeqIr.seuil == 60].merge(dfTfsdLadoumegue_1h, left_on='date_heure', right_on='date_heure_deb')

In [469]:
def ModulationTsfd(leq, tsfd, seuilLeq):
    """
    modifier le leq en fonction d'une valeurde tsfd, uniquement si le leq est superieur à un seuil
    """

,date_heure,leq_a,moy_pond_sur_k,note_cumul,nb_participants,IRSeuil,seuil,pression_base,pression_majoree,niveau_corrige,leq_majore,tsfd
0,2022-03-22 06:00:00,59.500861,9.461371,28.0,7.0,92.73,60,0.018883,0.017510,58.845266,62.195723,0.05
1,2022-03-22 07:00:00,61.970387,9.000000,36.0,7.0,77.07,60,0.025093,0.019339,59.708094,63.995212,0.04
2,2022-03-22 08:00:00,61.217734,8.922984,45.0,7.0,36.76,60,0.023010,0.008458,52.525244,61.768189,0.03
3,2022-03-22 09:00:00,59.341839,8.922984,45.0,7.0,81.65,60,0.018541,0.015138,57.580962,61.560341,0.04
4,2022-03-22 10:00:00,61.444734,8.589650,44.0,7.0,96.68,60,0.023619,0.022835,61.151467,64.310876,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...
436,2022-04-12 17:00:00,63.166018,5.306853,6.0,7.0,93.04,60,0.028796,0.026792,62.539412,65.874306,0.05
437,2022-04-12 18:00:00,62.295716,4.306853,10.0,7.0,99.10,60,0.026050,0.025816,62.217189,65.266930,0.05
438,2022-04-12 19:00:00,63.339963,3.306853,4.0,7.0,99.57,60,0.029378,0.029252,63.302533,66.331588,0.05
439,2022-04-12 20:00:00,63.685329,3.306853,4.0,7.0,85.84,60,0.030570,0.026241,62.359123,66.082953,0.06
